In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PineconeStore
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
# from pinecone import Pinecone

In [2]:
PINECONE_API_KEY = "pcsk_71uDMw_5woHgJLh2zFoX2B2b3g56JU7ZRnG3JchK45ZJh6izKad6nFYyZwVUZFuJacTygt"
PINECONE_API_ENV = "medical-chatbot"

In [3]:
def load_pdf(data):
    loader = DirectoryLoader(
        data, glob="*.pdf",loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [4]:
extracted_data = load_pdf("data/")

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)

In [8]:
len(text_chunks)

5860

In [4]:

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [5]:
embeddings = download_hugging_face_embeddings()

/tmp/ipykernel_41071/694119871.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [6]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [7]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
# from pinecone import Pinecone
# pc = Pinecone(api_key=PINECONE_API_KEY)  

 
# index_name=PINECONE_API_ENV
# index = pc.Index(index_name)  
# for i, t in zip(range(len(text_chunks)), text_chunks):
#     query_result = embeddings.embed_query(t.page_content)
#     index.upsert(
#     vectors=[
#         {
#             "id": str(i),  # Convert i to a string
#             "values": query_result, 
#             "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
#         }
#     ],
#     namespace="real" 
#     )
#     index.describe_index_stats() 

In [8]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)  
index_name = "medical-chatbot"
index = pc.Index(index_name)  


In [ ]:
# query = "What are Allergies"

# # Embed the query using your embedding model
# query_embedding = embeddings.embed_query(query) 
# results = index.query(
#     vector=query_embedding,
#     top_k=3,  # Number of closest matches to retrieve
#     include_metadata=True,  # Include metadata if necessary
#     namespace="real"  # Specify the namespace if applicable
# )


In [10]:
# print("Results:", results['matches'])


In [9]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [10]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [11]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [12]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [28]:
# from langchain.vectorstores import Pinecone

# # Initialize the Pinecone VectorStore
# vectordb = Pinecone(
#     index=index,  # Your Pinecone index instance
#     embeddings=embeddings,  # The embeddings object
#     text_key="text"  # Key for text in your documents
# ).as_retriever(search_kwargs={"k": 2})


In [24]:
from langchain.vectorstores import Pinecone as PineconeStore
# vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings).as_retriever()
vectordb = PineconeStore(index, embeddings,"text").as_retriever(search_kwargs={"k": 2})

In [25]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectordb,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [27]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :  Hemorrhoids are swollen veins in the rectum or anus that can cause pain, itching, and bleeding. They are a common condition that can be caused by straining during bowel movements, pregnancy, obesity, and other factors. Treatment options for hemorrhoids include home remedies such as witch hazel or ice packs, over-the-counter medications like ibuprofen or acetaminophen, and surgery in severe cases.
